In [117]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf
import tensorflow.keras as keras

Data Pipeline

In [58]:
from pathlib import Path
import tarfile
import urllib.request

def load_housing_data():
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True ,exist_ok=True)
        url = "https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pd.read_csv(Path("datasets/housing.csv"))        

In [59]:
housing = load_housing_data()

In [60]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [61]:
dataset_path = Path("datasets/housing.csv")
dataset_line = tf.data.TextLineDataset(str(dataset_path))
record_defaults = [tf.constant([0.], dtype=tf.float32)] * 7 + [tf.constant([0.], dtype=tf.float32)] * 2 + [tf.constant([], dtype=tf.string)] 

In [62]:
def parserx(line):
    x = tf.io.decode_csv(line, record_defaults=record_defaults)
    features1 = tf.stack(x[:8])
    features2 = tf.stack(tf.concat([x[0:2], x[3:5]], axis=0))
    return features1 , features2
def parsery(line):
    x = tf.io.decode_csv(line, record_defaults=record_defaults)
    return tf.stack(x[8])

In [63]:
data_x = dataset_line.skip(1).map(parserx)
data_y = dataset_line.skip(1).map(parsery)

In [64]:
data_x_wide_array = tf.TensorArray(tf.float32, size = 0 , dynamic_size=True)
data_x_deep_array = tf.TensorArray(tf.float32, size = 0 , dynamic_size=True)
data_y_array = tf.TensorArray(tf.float32, size = 0, dynamic_size = True)

In [65]:
for x, y in data_x:
    data_x_deep_array = data_x_deep_array.write(data_x_deep_array.size(), x)
    data_x_wide_array = data_x_wide_array.write(data_x_wide_array.size(), y)

In [66]:
train_x_deep = data_x_deep_array.stack()
train_x_wide = data_x_wide_array.stack()

In [67]:
for f in data_y:
    data_y_array = data_y_array.write(data_y_array.size(), f)

In [68]:
train_y = data_y_array.stack()

In [69]:
tf.reshape(train_y , (train_y.shape[0], 1))

<tf.Tensor: shape=(20640, 1), dtype=float32, numpy=
array([[452600.],
       [358500.],
       [352100.],
       ...,
       [ 92300.],
       [ 84700.],
       [ 89400.]], dtype=float32)>

## Utility Functions

In [140]:
from pathlib import Path
from time import strftime

def get_run_logdir(root_logdir='my_logs'):
  return Path(root_logdir)/strftime('rin_%Y_%m_%d_%H_%M_%S')

run_logdir = get_run_logdir()

## Model:

In [70]:
class RTGaussianRegNoiseLayer(tf.keras.layers.Layer):
  def __init__(self,units, stddev ,activation = None, **kwargs):
    super().__init__(**kwargs)
    self.stddev = stddev
    self.units = units
    self.activation = tf.keras.activations.get(activation)

  def build(self, batch_input_shape):

    self.kernel = self.add_weight(name = 'kernel',
                                  shape = [batch_input_shape[-1], self.units],
                                  initializer = tf.keras.initializers.GlorotNormal())
    self.bias = self.add_weight(name = 'bias',
                                shape = [self.units],
                                initializer = tf.keras.initializers.Zeros())

  def call(self, X , training = False):
    self.batch_size = tf.shape(X)[0]
    noise = tf.random.normal(shape = [self.batch_size, self.units], stddev = self.stddev)
    if training :
      if self.activation is not None:
        return self.activation(tf.matmul(X, self.kernel) + self.bias) + noise
      else:
        return tf.matmul(X, self.kernel) + self.bias + noise
    else:
      if self.activation is not None:
        return self.activation(tf.matmul(X, self.kernel) + self.bias)
      else:
        return tf.matmul(X, self.kernel) + self.bias

  def get_config(self):
    base_config = super().get_config()
    return {**base_config , 'units': self.units ,
            'activation' : tf.keras.activations.serialize(self.activation)}

In [119]:
@tf.keras.utils.register_keras_serializable()
class DualIOGaussianRT(tf.keras.Model):
 def __init__(self, architecture, keen_units, layer_depth, g_units , **kwargs):
        super().__init__(**kwargs)
        self.architecture = architecture
        self.keen_units = keen_units
        self.g_units = g_units
        self.layer_depth = layer_depth
        
        self.DeepFlow_ = [tf.keras.layers.Dense(units, activation = 'relu',
                                                kernel_initializer = tf.keras.initializers.GlorotNormal(),
                                                kernel_regularizer = tf.keras.regularizers.L2(0.01)) for units in architecture]
        self.BatchNormDeep_ = [tf.keras.layers.BatchNormalization() for _ in architecture]
        self.WideFlow = [tf.keras.layers.Dense(keen_units,
                                               activation = tf.keras.activations.relu,
                                               kernel_initializer = tf.keras.initializers.HeNormal(),
                                               kernel_regularizer= tf.keras.regularizers.L2(0.1)) for _ in range(self.layer_depth)]
        self.BatchNormWide_ = tf.keras.layers.BatchNormalization()
        self.GRNlayer  = RTGaussianRegNoiseLayer(g_units, 0.1, activation = 'relu')
        self.concatenate_ = tf.keras.layers.Concatenate()
        self.output_1 = tf.keras.layers.Dense(1,name='output_1', activation = tf.keras.activations.linear,
                                              kernel_initializer = tf.keras.initializers.GlorotNormal())
        self.output_2 = tf.keras.layers.Dense(1,name = 'output_2', activation = tf.keras.activations.linear,
                                              kernel_initializer = tf.keras.initializers.GlorotNormal())
        
 def call(self, input):
        x_deep , x_wide = input[0] , input[1]
        
       
        for layer, norm in zip(self.DeepFlow_, self.BatchNormDeep_):
            x_deep = layer(x_deep)
            x_deep = norm(x_deep)
        for l in self.WideFlow:
            x_wide = l(x_wide)
        x_wide = self.BatchNormWide_(x_wide)
        x_wide = self.GRNlayer(x_wide)
        x = self.concatenate_([x_deep, x_wide])
        out_1 = self.output_1(x)
        out_2 = self.output_2(x_deep)
        return {'output_1': out_1, 'output_2': out_2}

 def get_config(self):
        base_config = super().get_config()
        base_config.update({'architecture':self.architecture,
                            'keen_units':self.keen_units,
                            'g_units':self.g_units,
                            'layer_depth':self.layer_depth})
        return base_config

In [120]:
import keras_tuner as kt
class DualIOGaussianRTTuner(kt.HyperModel):
    def build(self, hp):
        architecture_depth= hp.Int('architecture_depth', min_value=1, max_value=9)
        architecture = [hp.Int(f'architecture_{i}', min_value=8, max_value=256, step=32) for i in range(architecture_depth)]
        keen_units = hp.Int('keen_units', min_value=8, max_value=128, step=32)
        g_units = hp.Int('g_units', min_value=8, max_value=128, step=32)
        layer_depth = hp.Int('layer_depth', min_value=1, max_value=3)
        learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        optimizer = hp.Choice('optimizer',  values = ['adam', 'rmsprop'])
        if optimizer == 'adam':
            optimizer = tf.keras.optimizers.Adam(learning_rate)
        else:
            optimizer = tf.keras.optimizers.RMSprop(learning_rate)

        model = DualIOGaussianRT(architecture, keen_units, layer_depth, g_units)
        model.compile(loss = {'output_1':tf.keras.losses.MeanSquaredError(),
                              'output_2':tf.keras.losses.MeanSquaredError()},
                      optimizer = optimizer,
                      metrics = {'output_1':[tf.keras.metrics.RootMeanSquaredError()],
                                 'output_2':[tf.keras.metrics.RootMeanSquaredError()]})          
        return model

In [121]:
objective = kt.Objective('val_loss', direction='min')
HyperModelTuner = kt.RandomSearch(hypermodel=DualIOGaussianRTTuner(),
                                  objective=objective,
                                  max_trials=20,
                                  overwrite = True,
                                  directory='tuner_dir',
                                  project_name='DualIOGaussianRT',
                                  seed=42,
                                  max_consecutive_failed_trials=5)

In [122]:
HyperModelTuner.search([train_x_deep, train_x_wide], {'output_1' : train_y, 'output_2': train_y}, epochs=30, validation_split=0.2)

Trial 20 Complete [00h 00m 40s]
val_loss: 12233279488.0

Best val_loss So Far: 9894522880.0
Total elapsed time: 00h 11m 31s


In [123]:
HyperModelTuner.get_best_hyperparameters()[0].values

{'architecture_depth': 5,
 'architecture_0': 168,
 'keen_units': 104,
 'g_units': 40,
 'layer_depth': 3,
 'learning_rate': 0.0004827322539144281,
 'optimizer': 'adam',
 'architecture_1': 168,
 'architecture_2': 136,
 'architecture_3': 200,
 'architecture_4': 8,
 'architecture_5': 200,
 'architecture_6': 168,
 'architecture_7': 72,
 'architecture_8': 232}

In [124]:
lr = HyperModelTuner.get_best_hyperparameters()[0].values['learning_rate']
architecture = [HyperModelTuner.get_best_hyperparameters()[0].values[f'architecture_{i}'] for i in range(HyperModelTuner.get_best_hyperparameters()[0].values['architecture_depth'])]
ku = HyperModelTuner.get_best_hyperparameters()[0].values['keen_units']
gu = HyperModelTuner.get_best_hyperparameters()[0].values['g_units']
ld = HyperModelTuner.get_best_hyperparameters()[0].values['layer_depth']

In [125]:
model = DualIOGaussianRT(architecture, ku, ld, gu)

In [126]:
model.compile(loss = {'output_1':tf.keras.losses.MeanSquaredError(),
                        'output_2':tf.keras.losses.MeanSquaredError()},
                optimizer = tf.keras.optimizers.Adam(lr),
                metrics = {'output_1':[tf.keras.metrics.RootMeanSquaredError()],
                             'output_2':[tf.keras.metrics.RootMeanSquaredError()]})

In [141]:
model.fit([train_x_deep, train_x_wide], {'output_1':train_y, 'output_2':train_y}, epochs=100, validation_split=0.2 , callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath = 'best_model.keras',
                                                                                                                                         monitor = 'val_loss',
                                                                                                                                         save_best_only = True,
                                                                                                                                         save_weights_only = False,
                                                                                                                                         mode = 'min',
                                                                                                                                         verbose =1),
                                                                                                                                         tf.keras.callbacks.TensorBoard(run_logdir)])

Epoch 1/100
507/516 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8082180096.0000 - output_1_root_mean_squared_error: 63516.7266 - output_2_root_mean_squared_error: 63597.0820
Epoch 1: val_loss improved from inf to 9897696256.00000, saving model to best_model.keras
516/516 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 8079166464.0000 - output_1_root_mean_squared_error: 63504.7695 - output_2_root_mean_squared_error: 63585.7109 - val_loss: 9897696256.0000 - val_output_1_root_mean_squared_error: 70322.4219 - val_output_2_root_mean_squared_error: 70373.6797
Epoch 2/100
505/516 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7695391232.0000 - output_1_root_mean_squared_error: 61938.3555 - output_2_root_mean_squared_error: 62098.0977
Epoch 2: val_loss improved from 9897696256.00000 to 9325228032.00000, saving model to best_model.keras
516/516 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 7699109376.0000 - output_1_root_mean_squared_error: 61954.2070 - output_2_root_mean_squared_error: 62112.6016 - val_loss: 93252

In [142]:
best_model = tf.keras.models.load_model('best_model.keras')

In [143]:
y_pred = model.predict([train_x_deep, train_x_wide])

645/645 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [144]:
y_pred_main , y_pred_aux = y_pred['output_1'] , y_pred['output_2']

In [146]:
y_pred_main[:30] , train_y[0:20]

(array([[458213.53 ],
        [493379.25 ],
        [447631.12 ],
        [350634.22 ],
        [287264.72 ],
        [302420.97 ],
        [296541.28 ],
        [394601.25 ],
        [219321.23 ],
        [346142.22 ],
        [212264.17 ],
        [357902.44 ],
        [195709.31 ],
        [242397.53 ],
        [220331.48 ],
        [148018.97 ],
        [137817.44 ],
        [182326.23 ],
        [115894.88 ],
        [136772.1  ],
        [ 85833.26 ],
        [ 90175.65 ],
        [123080.9  ],
        [118834.7  ],
        [148742.05 ],
        [102133.87 ],
        [147222.33 ],
        [111069.58 ],
        [119431.87 ],
        [114971.086]], dtype=float32),
 <tf.Tensor: shape=(20,), dtype=float32, numpy=
 array([452600., 358500., 352100., 341300., 342200., 269700., 299200.,
        241400., 226700., 261100., 281500., 241800., 213500., 191300.,
        159200., 140000., 152500., 155500., 158700., 162900.],
       dtype=float32)>)